## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [9]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

Libraries imported.


In [10]:
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4

     |████████████████████████████████| 5.8MB 3.0MB/s eta 0:00:01
     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:011


<b>Use the Notebook to build the code to scrape the following Wikipedia page, 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below</b>

In [11]:
#!conda install -c conda-forge lxml --yes
#!conda install -c conda-forge BeautifulSoup4 --yes
#!conda install -c conda-forge html5lib --yes
import html5lib
import lxml 

toronto_neighbourhoods_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]
toronto_neighbourhoods_df.head()
# jan_march = toronto_neigh_df[3].iloc[:]

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<B>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.<b>

In [12]:
T_neigh_df_clean = toronto_neighbourhoods_df.set_index("Borough")
T_neigh_df_clean.drop("Not assigned", axis = 0,inplace=True)
T_neigh_df_clean.reset_index(inplace=True)
T_neigh_df_clean

,Borough,Postcode,Neighbourhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Harbourfront
3,Downtown Toronto,M5A,Regent Park
4,North York,M6A,Lawrence Heights
...,...,...,...
206,Etobicoke,M8Z,Kingsway Park South West
207,Etobicoke,M8Z,Mimico NW
208,Etobicoke,M8Z,The Queensway West
209,Etobicoke,M8Z,Royal York South West


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [13]:
T_neigh_df_clean.loc[T_neigh_df_clean['Neighbourhood'] == 'Not assigned','Neighbourhood'] = np.nan
T_neigh_df_clean[T_neigh_df_clean["Neighbourhood"] == "Not assigned"]
T_neigh_df_clean["Neighbourhood"] = T_neigh_df_clean["Neighbourhood"].fillna(T_neigh_df_clean["Borough"])

In [6]:
T_neigh_df_clean.head(10)

,Borough,Postcode,Neighbourhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Harbourfront
3,Downtown Toronto,M5A,Regent Park
4,North York,M6A,Lawrence Heights
5,North York,M6A,Lawrence Manor
6,Queen's Park,M7A,Queen's Park
7,Etobicoke,M9A,Islington Avenue
8,Scarborough,M1B,Rouge
9,Scarborough,M1B,Malvern


### More than one neighborhood can exist in one postal code area.Two rows will be combined into one row

In [14]:
T_neigh_df_clean_concat = T_neigh_df_clean.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
T_neigh_df_clean_concat

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [15]:
T_neigh_df_clean_concat.shape

(103, 3)

<B> *************This Ends the first part of the homework******************</B>
### Use the Geocoder package or the csv file to create the following dataframe:

In [16]:
# Reading Geo codes using CSV

toronto_geocodes = pd.read_csv('http://cocl.us/Geospatial_data')
toronto_geocodes

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [ ]:
# pip install geocoder
# !conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
 # g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
 # lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

#lat_lng_coords

### Join two DFs - Neighbourhoods and GeoCodes

In [23]:
T_neigh_df_full = T_neigh_df_clean_concat.set_index('Postcode').join(toronto_geocodes.set_index('Postal Code'))
T_neigh_df_full.reset_index(inplace = True)
T_neigh_df_full

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [24]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<b>Explore and cluster the neighborhoods in Toronto</b>

In [28]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(T_neigh_df_full['Latitude'], T_neigh_df_full['Longitude'], T_neigh_df_full['Borough'], T_neigh_df_full['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Using Foursquare API to get the Neighbourhood Venues

In [29]:
CLIENT_ID = '0MWLXE21XWIGAFGOXET21G53OQXRSGUSCV5DFZWDTA5ESPJ5' # your Foursquare ID
CLIENT_SECRET = 'VPTKIT4VQCH5VDZS0OXN2MJIX2JE0LP1ZRADXFQTUUXE55ZS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0MWLXE21XWIGAFGOXET21G53OQXRSGUSCV5DFZWDTA5ESPJ5
CLIENT_SECRET:VPTKIT4VQCH5VDZS0OXN2MJIX2JE0LP1ZRADXFQTUUXE55ZS


In [36]:
Toronto_data = T_neigh_df_full[T_neigh_df_full['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Toronto_data.head()
print(Toronto_data.loc[0, 'Neighbourhood'])
neighbourhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = Toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))


Rosedale
Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [38]:
# type your answer here
LIMIT = 5
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, radius, LIMIT)
print(url)

https://api.foursquare.com/v2/venues/search?client_id=0MWLXE21XWIGAFGOXET21G53OQXRSGUSCV5DFZWDTA5ESPJ5&client_secret=VPTKIT4VQCH5VDZS0OXN2MJIX2JE0LP1ZRADXFQTUUXE55ZS&ll=43.6795626,-79.37752940000001&v=20180605&radius=500&limit=5


In [39]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d6c150d89b06a0039817c8f'},
 'response': {'venues': [{'id': '4aff2d47f964a520743522e3',
    'name': 'Rosedale Park',
    'location': {'address': '38 Scholfield Ave.',
     'crossStreet': 'at Edgar Ave.',
     'lat': 43.68232820227814,
     'lng': -79.37893434347683,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.68232820227814,
       'lng': -79.37893434347683}],
     'distance': 327,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['38 Scholfield Ave. (at Edgar Ave.)',
      'Toronto ON',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e7941735',
      'name': 'Playground',
      'pluralName': 'Playgrounds',
      'shortName': 'Playground',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/playground_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1567364365',
    'hasPerk': False},
   {'id': '4bae2150

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
Downtown_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )



Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [43]:
print(Downtown_venues.shape)
Downtown_venues.head()

(90, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


### Count of Venues by Neighbourhoods

In [45]:
Downtown_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",5,5,5,5,5,5
Berczy Park,5,5,5,5,5,5
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",5,5,5,5,5,5
"Cabbagetown, St. James Town",5,5,5,5,5,5
Central Bay Street,5,5,5,5,5,5
"Chinatown, Grange Park, Kensington Market",5,5,5,5,5,5
Christie,5,5,5,5,5,5
Church and Wellesley,5,5,5,5,5,5
"Commerce Court, Victoria Hotel",5,5,5,5,5,5


In [47]:
print('There are {} uniques categories.'.format(len(Downtown_venues['Venue Category'].unique())))

There are 50 uniques categories.


## Analyze Each Neighborhood

In [49]:
# one hot encoding
downtown_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add Neighbourhood column back to dataframe
downtown_onehot['Neighbourhood'] = Downtown_venues['Neighbourhood'] 

# move Neighbourhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Bakery,Beer Bar,Breakfast Spot,...,Ramen Restaurant,Restaurant,Salad Place,Spa,Steakhouse,Supermarket,Thai Restaurant,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [50]:
downtown_onehot.shape

(90, 51)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [51]:
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Bakery,Beer Bar,Breakfast Spot,...,Ramen Restaurant,Restaurant,Salad Place,Spa,Steakhouse,Supermarket,Thai Restaurant,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.2
1,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.2
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Cabbagetown, St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"Chinatown, Grange Park, Kensington Market",0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,...,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0
8,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"Design Exchange, Toronto Dominion Centre",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<b>Let's print each neighborhood along with the top 5 most common venues</b>

In [53]:
num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                           venue  freq
0  Vegetarian / Vegan Restaurant   0.2
1                     Steakhouse   0.2
2                   Concert Hall   0.2
3                          Hotel   0.2
4                          Plaza   0.2


----Berczy Park----
                           venue  freq
0  Vegetarian / Vegan Restaurant   0.2
1                   Liquor Store   0.2
2                     Steakhouse   0.2
3                         Museum   0.2
4                   Concert Hall   0.2


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0             Airport   0.2
1    Airport Terminal   0.2
2  Airport Food Court   0.2
3     Harbor / Marina   0.2
4      Airport Lounge   0.2


----Cabbagetown, St. James Town----
                 venue  freq
0    Indian Restaurant   0.2
1   Italian Restaurant   0.2
2  Japanese Restaurant   0.2
3                Diner   0.2
4        

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Above is a function to sort the venues in descending order.

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
Neighbourhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    Neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

Neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Vegetarian / Vegan Restaurant,Hotel,Steakhouse,Concert Hall,Plaza,Burrito Place,Diner,Dessert Shop,Dance Studio,Creperie
1,Berczy Park,Vegetarian / Vegan Restaurant,Steakhouse,Concert Hall,Liquor Store,Museum,Café,Diner,Dessert Shop,Dance Studio,Creperie
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport,Airport Lounge,Airport Terminal,Harbor / Marina,Airport Food Court,Coffee Shop,Grocery Store,Gastropub,Diner,Dessert Shop
3,"Cabbagetown, St. James Town",Diner,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Café,Dessert Shop,Dance Studio,Creperie
4,Central Bay Street,Coffee Shop,Gastropub,Italian Restaurant,Vegetarian / Vegan Restaurant,Café,Diner,Dessert Shop,Dance Studio,Creperie,Concert Hall


## Cluster the neighbourhoods

In [64]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 4, 1, 1, 1, 0, 3, 3], dtype=int32)

In [68]:

# Neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each Neighbourhood
downtown_merged = downtown_merged.join(Neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Trail,Building,Playground,Gastropub,Diner,Dessert Shop,Dance Studio,Creperie,Concert Hall
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,4,Diner,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Café,Dessert Shop,Dance Studio,Creperie
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Theme Restaurant,Dance Studio,Ramen Restaurant,Breakfast Spot,Bubble Tea Shop,Vegetarian / Vegan Restaurant,Café,Gastropub,Diner,Dessert Shop
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,3,Gym / Fitness Center,Spa,Restaurant,Bakery,Coffee Shop,Café,Gastropub,Diner,Dessert Shop,Dance Studio
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Pizza Place,Burrito Place,Comic Shop,Clothing Store,Plaza,Gastropub,Diner,Dessert Shop,Dance Studio,Creperie


#### Visuaization of the clusters

In [70]:

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters